# Imports

In [1]:
import math
import pandas as pd
import numpy as np
import glob
import datetime
pd.options.mode.chained_assignment = None

# Table of Contents
 <p><div class="lev1"><a href="#Task-1.-Compiling-Ebola-Data"><span class="toc-item-num">Task 1.&nbsp;&nbsp;</span>Compiling Ebola Data</a></div>
 <div class="lev1"><a href="#Task-2.-RNA-Sequences"><span class="toc-item-num">Task 2.&nbsp;&nbsp;</span>RNA Sequences</a></div>
 <div class="lev1"><a href="#Task-3.-Class-War-in-Titanic"><span class="toc-item-num">Task 3.&nbsp;&nbsp;</span>Class War in Titanic</a></div></p>

In [2]:
DATA_FOLDER = 'Data/ebola' # Use the data folder provided in Tutorial 02 - Intro to Pandas.

## Task 1. Compiling Ebola Data

The `DATA_FOLDER/ebola` folder contains summarized reports of Ebola cases from three countries (Guinea, Liberia and Sierra Leone) during the recent outbreak of the disease in West Africa. For each country, there are daily reports that contain various information about the outbreak in several cities in each country.

Use pandas to import these data files into a single `Dataframe`.
Using this `DataFrame`, calculate for *each country*, the *daily average per month* of *new cases* and *deaths*.
Make sure you handle all the different expressions for *new cases* and *deaths* that are used in the reports.

## Constants

In [3]:
DEFAULT_NAME_COLUMN_DATE = 'Date'
DEFAULT_NAME_COLUMN_DESCRIPTION = 'Description'
DEFAULT_NAME_COLUMN_TOTAL = 'Totals'
DEFAULT_NAME_COLUMN_COUNTRY = 'Country'

DEFAULT_NAME_ROW_DESCRIPTION_NEW_CASES = 'New cases'
DEFAULT_NAME_ROW_DESCRIPTION_NEW_DEATHS = 'New deaths'

DEFAULT_DATA_FORMAT = '%Y-%m-%d'

## Auxiliary functions

In [4]:
def sanitize_date(date_col, country_index):
    date_list = list()
    for i, date in date_col.iteritems():
        date_rep = None
        try:
            date_rep = datetime.datetime.strptime(date, data_format_list[country_index][0])
        except ValueError:
            date_rep = datetime.datetime.strptime(date, data_format_list[country_index][1])
        date_list.append(date_rep.strftime(DEFAULT_DATA_FORMAT))
    return np.array(date_list)

def sum_string_list(ls):
    temp_sum = 0
    for elem in ls:
        temp_sum += float(elem)
    return temp_sum
    
def estimate_daily_new_deaths(np_list):
    last_known_n = -1
    final_list = list()
    for i, elem in np_list.iteritems():
        if (math.isnan(elem)):
            final_list.append(0)
        else:
            if (last_known_n == -1):
                final_list.append(0)
            else:
                final_list.append(elem - last_known_n)
            last_known_n = elem
    return np.array(final_list)

In [5]:
path_list = [DATA_FOLDER + '/guinea_data', DATA_FOLDER + '/liberia_data', DATA_FOLDER + '/sl_data']
country_list = ['Guinea', 'Liberia', 'Sierra Leone']
wanted_column_list = [['Date', 'Description', 'Totals'], ['Date', 'Variable', 'National'], ['date', 'variable', 'National']]
data_format_list = [['%Y-%m-%d', '%y-%m-%d'], ['%m/%d/%Y', '%m/%d/%y'], ['%Y-%m-%d', '%y-%m-%d']]
dataframe_list = [pd.DataFrame(), pd.DataFrame(), pd.DataFrame()]    

for i in range (0,3):
    temp_list = []
    all_files = glob.glob(path_list[i] + '/*.csv')
    for file in all_files:
        temp_df = pd.read_csv(file)
        temp_list.append(temp_df)
    dataframe_list[i] = pd.concat(temp_list)
    dataframe_list[i]['Country'] = country_list[i]
    dataframe_list[i][wanted_column_list[i][0]] = sanitize_date(dataframe_list[i][wanted_column_list[i][0]], i)
    dataframe_list[i][wanted_column_list[i][0]] = pd.to_datetime(dataframe_list[i][wanted_column_list[i][0]])
    dataframe_list[i].sort_values(by=wanted_column_list[i][0])


## Parsing functions

In [7]:
def parse_guinea_files():
    
    # Select certain column from the CSV formatted files
    parsed_guinea_df = dataframe_list[0][['Date', 'Description', 'Totals']]
    wanted_row_list = ['Total new cases registered so far', 'New deaths registered', 'New deaths registered today']
    
    # Select all the rows that match desired names for 'Description' column
    parsed_guinea_df = parsed_guinea_df[parsed_guinea_df['Description'].isin(wanted_row_list)]
    
    # Rename every row value for 'Description' column 
    parsed_guinea_df.loc[parsed_guinea_df['Description'] == wanted_row_list[0], 'Description'] = DEFAULT_NAME_ROW_DESCRIPTION_NEW_CASES
    parsed_guinea_df.loc[parsed_guinea_df['Description'] == wanted_row_list[1], 'Description'] = DEFAULT_NAME_ROW_DESCRIPTION_NEW_DEATHS
    parsed_guinea_df.loc[parsed_guinea_df['Description'] == wanted_row_list[2], 'Description'] = DEFAULT_NAME_ROW_DESCRIPTION_NEW_DEATHS
    
    # Add 'Country' column to dataframe
    parsed_guinea_df[DEFAULT_NAME_COLUMN_COUNTRY] = 'Guinea'
    return parsed_guinea_df

def parse_liberia_files():
    
    # Select certain column from the CSV formatted files
    parsed_liberia_df = dataframe_list[1][['Date', 'Variable', 'National']]
    wanted_row_list = ['New Case/s (Suspected)', 'New Case/s (Probable)', 'New case/s (confirmed)', 'Newly reported deaths']
    
    # Create temporary dataframe for new case entries (with every row that matches the desired 'Variables' column value)
    parsed_liberia_df_new_cases = parsed_liberia_df[parsed_liberia_df['Variable'].isin(wanted_row_list[0:3])]
    
    # Sum all of the values for 'Probable', 'Variable' and 'Confirmed' new cases
    parsed_liberia_df_new_cases = parsed_liberia_df_new_cases.groupby('Date').sum()
    parsed_liberia_df_new_cases['Date'] = parsed_liberia_df_new_cases.index
    parsed_liberia_df_new_cases['Variable'] = DEFAULT_NAME_ROW_DESCRIPTION_NEW_CASES
    
    # Create temporary dataframe for new deaths entries
    parsed_liberia_df_new_deaths = parsed_liberia_df[parsed_liberia_df['Variable'] == wanted_row_list[3]]
    parsed_liberia_df_new_deaths['Variable'] = DEFAULT_NAME_ROW_DESCRIPTION_NEW_DEATHS
    
    # Create new dataframe with new deaths and cases (concatenating the two temporary)
    parsed_liberia_df = pd.concat([parsed_liberia_df_new_cases, parsed_liberia_df_new_deaths])
    parsed_liberia_df[DEFAULT_NAME_COLUMN_COUNTRY] = 'Liberia'
    
    parsed_liberia_df.rename(columns={'Date':DEFAULT_NAME_COLUMN_DATE, 'Variable':DEFAULT_NAME_COLUMN_DESCRIPTION, 'National':DEFAULT_NAME_COLUMN_TOTAL}, inplace=True)
    
    return parsed_liberia_df

def parse_sierra_files():
    
    parsed_sierra_df = dataframe_list[2][['date', 'variable', 'National']]
    wanted_row_list = ['new_noncase', 'new_suspected', 'new_probable', 'new_confirmed', 'death_confirmed', 'death_probable', 'death_suspected']
    
    # Create temporary dataframe for new case entries (with every row that matches the desired 'variables' column value)
    parsed_sierra_df_new_cases = parsed_sierra_df[parsed_sierra_df['variable'].isin(wanted_row_list[0:4])]
    
    # Sum all of the values for 'Probable', 'Variable', 'Confirmed' and 'Noncase' new cases
    parsed_sierra_df_new_cases = parsed_sierra_df_new_cases.groupby('date')['National'].apply(lambda x: sum_string_list(x)).to_frame()
    parsed_sierra_df_new_cases['date'] = parsed_sierra_df_new_cases.index
    parsed_sierra_df_new_cases['variable'] = DEFAULT_NAME_ROW_DESCRIPTION_NEW_CASES
    
    # Create temporary dataframe for new deaths' accumulator entries
    parsed_sierra_df_new_deaths_cum = parsed_sierra_df[parsed_sierra_df['variable'].isin(wanted_row_list[4:7])]
    
    # Sum all of the values for 'Probable', 'Suspected' and 'Confirmed' new death accumulators
    parsed_sierra_df_new_deaths_cum = parsed_sierra_df_new_deaths_cum.groupby('date')['National'].apply(lambda x: sum_string_list(x)).to_frame()
    parsed_sierra_df_new_deaths_cum['date'] = parsed_sierra_df_new_deaths_cum.index
    parsed_sierra_df_new_deaths_cum['variable'] = 'New deaths accumulator'
    
    # 
    parsed_sierra_df_new_deaths_cum['National'] = estimate_daily_new_deaths(parsed_sierra_df_new_deaths_cum['National'])
    parsed_sierra_df_new_deaths_cum['variable'] = DEFAULT_NAME_ROW_DESCRIPTION_NEW_DEATHS
    
    #
    parsed_sierra_df = pd.concat([parsed_sierra_df_new_cases, parsed_sierra_df_new_deaths_cum])
    parsed_sierra_df[DEFAULT_NAME_COLUMN_COUNTRY] = "Sierra"
    
    parsed_sierra_df.rename(columns={'date':DEFAULT_NAME_COLUMN_DATE, 'variable':DEFAULT_NAME_COLUMN_DESCRIPTION, 'National':DEFAULT_NAME_COLUMN_TOTAL}, inplace=True)
    
    return parsed_sierra_df

In [8]:
complete_df = pd.concat([parse_guinea_files(), parse_liberia_files(), parse_sierra_files()])
complete_df.reset_index(inplace=True, drop=True)
complete_df

,Country,Date,Description,Totals
0,Guinea,2014-08-04,New cases,9
1,Guinea,2014-08-04,New deaths,2
2,Guinea,2014-08-26,New cases,28
3,Guinea,2014-08-26,New deaths,5
4,Guinea,2014-08-27,New cases,22
5,Guinea,2014-08-27,New deaths,2
6,Guinea,2014-08-30,New cases,24
7,Guinea,2014-08-30,New deaths,5
8,Guinea,2014-08-31,New cases,46
9,Guinea,2014-08-31,New deaths,3


## Task 2. RNA Sequences

In the `DATA_FOLDER/microbiome` subdirectory, there are 9 spreadsheets of microbiome data that was acquired from high-throughput RNA sequencing procedures, along with a 10<sup>th</sup> file that describes the content of each. 

Use pandas to import the first 9 spreadsheets into a single `DataFrame`.
Then, add the metadata information from the 10<sup>th</sup> spreadsheet as columns in the combined `DataFrame`.
Make sure that the final `DataFrame` has a unique index and all the `NaN` values have been replaced by the tag `unknown`.

In [5]:
# Write your answer here

## Task 3. Class War in Titanic

Use pandas to import the data file `Data/titanic.xls`. It contains data on all the passengers that travelled on the Titanic.

In [ ]:
from IPython.core.display import HTML
HTML(filename=DATA_FOLDER+'/titanic.html')

For each of the following questions state clearly your assumptions and discuss your findings:
1. Describe the *type* and the *value range* of each attribute. Indicate and transform the attributes that can be `Categorical`. 
2. Plot histograms for the *travel class*, *embarkation port*, *sex* and *age* attributes. For the latter one, use *discrete decade intervals*. 
3. Calculate the proportion of passengers by *cabin floor*. Present your results in a *pie chart*.
4. For each *travel class*, calculate the proportion of the passengers that survived. Present your results in *pie charts*.
5. Calculate the proportion of the passengers that survived by *travel class* and *sex*. Present your results in *a single histogram*.
6. Create 2 equally populated *age categories* and calculate survival proportions by *age category*, *travel class* and *sex*. Present your results in a `DataFrame` with unique index.

In [ ]:
# Write your answer here